In [76]:
import pandas as pd
import sqlite3
from settings import DF_NAME
import dateparser
import warnings
warnings.filterwarnings("ignore")

def last_updated_to_datetime(row):
    timestamp_datetime = dateparser.parse(row.timestamp)
    settings = dict(RELATIVE_BASE=timestamp_datetime)
    last_updated_clean = row.last_updated.replace('Updated ', '').replace('moments', '0 minutes')

    # Ignore updates older than an hour for simplicity, will be removed later
    if 'minutes' in last_updated_clean:
        update_time = dateparser.parse(last_updated_clean, settings=settings)
    else:
        update_time = None

    row['update_time'] = update_time
    return row

def match_updates(df):
    df['match'] = None
    for i, row in df.iterrows():
        if not df.match[i]:
            deltas = (row['update_time'] - df.update_time)
            delta_filter = deltas.dt.total_seconds().abs().le(300)
            df.match[delta_filter] = i
    return df

def reduce_updates(df):
    reduced = df[df.last_updated.str.contains('minutes')]
    return reduced.drop_duplicates(subset=['location', 'current_count', 'match'])

conn = sqlite3.connect(DF_NAME)
df = pd.read_sql("select * from usage", conn)
df_update_time = df.apply(last_updated_to_datetime, axis=1)         
matched = match_updates(df_update_time)
reduced = reduce_updates(matched)

In [77]:
import plotly.express as px

reduced.max_count = pd.to_numeric(reduced.max_count)
reduced.current_count = pd.to_numeric(reduced.current_count)
reduced['capacity'] = reduced.current_count / reduced.max_count

fig = px.scatter(reduced[reduced.location.str.contains('Fitness')], x='timestamp', y='capacity', color='location')
fig.update_yaxes(range=(0,2))

In [84]:
group1 = reduced[reduced.location.str.contains('Fitness')].groupby('match').get_group(1)

group1.current_count.sum() / group1.max_count.sum()

0.7548387096774194

In [103]:
import numpy as np

fitness = reduced[reduced.location.str.contains('Fitness')]
fitness.groupby('match').agg(
    current_sum=pd.NamedAgg(column='current_count', aggfunc='sum'),
    max_sum=pd.NamedAgg(column='max_count', aggfunc='sum'),
    update_time=pd.NamedAgg(column='update_time', aggfunc='mean'),
)

,current_sum,max_sum,update_time
match,,,
1,117,155,2022-01-30 13:30:41.860945920
278,54,70,2022-01-30 15:09:53.811023872
283,42,40,2022-01-30 15:22:09.645491968
288,39,45,2022-01-30 15:30:09.645491968
314,75,70,2022-01-30 16:25:20.101494016
324,40,45,2022-01-30 16:38:03.478376960
325,50,40,2022-01-30 16:31:03.478376960
361,30,45,2022-01-30 17:46:58.013772032
362,134,110,2022-01-30 17:41:21.067044352
